# 🧠 Fine-tune T5-base to Generate JSON Tickets from Support Issues

This notebook trains a `t5-base` model using HuggingFace Transformers on a dataset of customer issue descriptions and structured ticket JSON outputs. It includes evaluation metrics like ROUGE and exact match.

In [1]:
# %pip install nltk rouge_score absl-py sentencepiece datasets transformers evaluate

In [2]:
# ✨ Import required libraries
import json
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import torch
import random
import evaluate
import numpy as np

/Users/aristideisingizwe/Documents/projects/sautidesk/sautidesk-model/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/aristideisingizwe/Documents/projects/sautidesk/sautidesk-model/venv/lib/python3.12/site-packages/ip

In [3]:
# ✨ Upload your JSON file
with open("../data/training_data.json", "r") as f:
    raw_data = json.load(f)

print(f"✅ Loaded {len(raw_data)} samples")

✅ Loaded 5000 samples


In [4]:
# ✨ Prepare dataset

def format_example(example):
    input_text = f"Generate ticket from: {example['text']}"
    target_text = json.dumps(example["label"])
    return {"input": input_text, "target": target_text}

formatted_data = [format_example(example) for example in raw_data]
dataset = Dataset.from_list(formatted_data)

# Split into train/test
dataset = dataset.train_test_split(test_size=0.1)
train_data = dataset["train"]
test_data = dataset["test"]

In [5]:
# ✨ Tokenize
import sentencepiece

model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)

MAX_INPUT_LEN = 256
MAX_TARGET_LEN = 512

def tokenize_function(example):
    model_inputs = tokenizer(
        example["input"],
        max_length=MAX_INPUT_LEN,
        truncation=True,
        padding="max_length"
    )
    labels = tokenizer(
        example["target"],
        max_length=MAX_TARGET_LEN,
        truncation=True,
        padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_test = test_data.map(tokenize_function, batched=True)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Map: 100%|██████████| 500/500 [00:00<00:00, 914.18 examples/s]


In [6]:
# ✨ Load model
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [7]:
# install 'nltk', 'absl-py', 'rouge_score'

In [8]:
# ✨ Define metrics
rouge = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    exact_matches = [int(p == l) for p, l in zip(decoded_preds, decoded_labels)]
    exact_match_score = np.mean(exact_matches)

    result["exact_match"] = exact_match_score
    return result

In [9]:
# ✨ Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-ticket-output",
    # evaluation_strategy="epoch",
    logging_dir="./logs",
    per_device_train_batch_size=1, # it was 4 before
    per_device_eval_batch_size=1, # it was 4 before
    num_train_epochs=1,
    weight_decay=0.01,
    save_total_limit=2,
    save_strategy="epoch",
    logging_steps=20,
    report_to="none"
)

In [10]:
# ✨ Trainer setup
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/var/folders/_2/12_yd5k13j34v8mhrlwz176r0000gn/T/ipykernel_1017/1944632754.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [11]:
# ✨ Train!
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
20,5.529200
40,0.809600
60,0.476400
80,0.340900
100,0.249100
120,0.211100
140,0.163700
160,0.143900
180,0.128500
200,0.120100


TrainOutput(global_step=4500, training_loss=0.08109420846568213, metrics={'train_runtime': 33373.2692, 'train_samples_per_second': 0.135, 'train_steps_per_second': 0.135, 'total_flos': 1370155253760000.0, 'train_loss': 0.08109420846568213, 'epoch': 1.0})

In [12]:
# ✅ Save model
model.save_pretrained("models/t5-ticket-model")
tokenizer.save_pretrained("models/t5-ticket-model")

('models/t5-ticket-model/tokenizer_config.json',
 'models/t5-ticket-model/special_tokens_map.json',
 'models/t5-ticket-model/spiece.model',
 'models/t5-ticket-model/added_tokens.json')

## ✅ Predict Example
Use the model to generate a ticket from text:

In [13]:
def predict_ticket(issue_text):
    input_text = f"Generate ticket from: {issue_text}"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    output = model.generate(**inputs, max_length=512)
    return tokenizer.decode(output[0], skip_special_tokens=True)

predict_ticket("My electricity has been out since last night. Please help.")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
/Users/aristideisingizwe/Documents/projects/sautidesk/sautidesk-model/venv/lib/python3.12/site-packages/transformers/pytorch_utils.py:332: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)


'"title": "Electricity issue", "description": "My electricity has been out since last night.", "state": "RESOLVED", "priority": "HIGH", "assignedTo": null, "ownedBy": "user_1212", "organisation": "REG", "tags": "electricity", "source": "AI", "type": "SUGGESTION"'